<a href="https://colab.research.google.com/github/pycriador/PyNLTK/blob/main/%5BUpload%5D_PyNLTK_Busca_por_similaridade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Projeto de Abril

Um programa simples para encontrar similaridade em frases com base em um dicionário.

Primeiro, vamos importar as bibliotecas necessárias:

In [ ]:
import numpy as np
import pandas as pd
import unicodedata, re
import json
import os

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')
nltk.download('stopwords')

Em seguida, vamos importar as classes que vamos utilizar no projeto:

In [36]:
class check_similar():
       
    def normalize(self, palavra):
        palavra = palavra.replace(',',' ')
        palavra = palavra.replace('.',' ')
        palavra = palavra.replace(';',' ')
        palavra = palavra.replace(':',' ')
        palavra = palavra.replace('-',' ')
        nfkd = unicodedata.normalize('NFKD', palavra)
        palavra = u"".join([c for c in nfkd if not unicodedata.combining(c)])
        palavra = re.sub('[^a-zA-Z0-9 \\\]', '', palavra)
        palavra = palavra.upper()
        return palavra
    
    def carregar_excel(self, xlsx, sheet=None):
        xlsx = pd.ExcelFile(xlsx)
        self.base = pd.read_excel(xlsx,sheet)
        return self.base
    
    def criar_lista(self, coluna, normalizar=True, df=None):
        if not df.empty:
            resultado = df[coluna].dropna()
        else:
            resultado = self.base[coluna].dropna()
        resultado = resultado.unique()
        self.lstresultado = resultado.tolist()
        if normalizar:
            self.lstresultado = [self.normalize(x) for x in self.lstresultado]
        return self.lstresultado
      
        
    def check_similar(self, busca, porcentagem=50):
        self.porcentagem = porcentagem
        busca_normalizado = self.normalize(busca)
        
        self.resultado = {}
        lst_porcentagens = []
    
        for Y in self.lstresultado:
            #X = self.busca
            X = busca_normalizado
            
            # tokenization 
            X_list = word_tokenize(X)  
            Y_list = word_tokenize(Y)
  
            # sw contains the list of stopwords 
            sw = stopwords.words('portuguese')  
            l1 =[];l2 =[] 
  
            # remove stop words from the string 
            X_set = {w for w in X_list if not w in sw}  
            Y_set = {w for w in Y_list if not w in sw}
            
  
            # form a set containing keywords of both strings  
            rvector = X_set.union(Y_set)  
            for w in rvector: 
                if w in X_set: l1.append(1) # create a vector 
                else: l1.append(0) 
                if w in Y_set: l2.append(1) 
                else: l2.append(0) 
            c = 0
  
            # cosine formula  
            for i in range(len(rvector)): 
                c+= l1[i]*l2[i] 
                cosine = c / float((sum(l1)*sum(l2))**0.5)
            porcento = float("{:.3f}".format(cosine))*100
            
            if round(porcento, 0) > self.porcentagem:
                if round(porcento, 0) == 100:
                    lst_porcentagens=[]
                    return Y
                    break
                    
                self.resultado.setdefault(round(porcento, 1), []).append(Y)
                lst_porcentagens.append(round(porcento, 1))
                
        if len(self.resultado) == 1:
            return self.resultado[lst_porcentagens[-1]][0]

        if lst_porcentagens:
            lst_porcentagens.sort()
            return self.resultado

class Core():

    def __init__(self):
        self.df_base = []

    def files(self, path, extension):
        for file in os.listdir(path):
            if os.path.isfile(os.path.join(path, file)):
                if re.search(extension, file):
                    yield file

    def dir_list(self, path=None, extension=None):
        dir_list = []
        for path_info in path:
            dirs = self.files(path_info, extension)
            for dir_info in dirs:
                dir_list.append("%s/%s" % (path_info, dir_info))
        return dir_list

    def read(self, input=None, ies=None, clean_cache=None, encode=None):
        if clean_cache:
            self.df_base = []

        if encode:
          encoding = encode
        else:
          encoding = 'iso-8859-1'

        df = pd.read_csv(input, sep=";", encoding=encoding, low_memory=False)
        
        base_cols = df.columns.tolist()
        for col in base_cols:
            try:
                df[col] = df[col].str.strip()
            except Exception as e:
                pass
        self.df_base.append(df)

    def join(self):
        self.result = pd.concat(self.df_base, sort=False)
        self.result = self.result.replace(np.nan, '', regex=True)

    def export(self):
        return self.result #.values.tolist()
        #Output sem formatação
        #with pd.ExcelWriter(f'PANDAS.xlsx') as writer:
        #    self.result.to_excel(writer, sheet_name=sheet, index=False)


Limpando e criando a pasta para colocar os dicionários:

In [ ]:
import shutil

print('Limpando arquivos antigos')
try:
  shutil.rmtree('arquivos')
except Exception as e:
  pass
  #print(f'Erro: {e}')

print('Criando a pasta "arquivos"')
!mkdir arquivos

Informar o arquivo CSV com o dicionário que deseja utilizar (é possível escolher vários arquivos ao mesmo tempo):

In [ ]:
#Dicionário utilizado
#http://blog.mds.gov.br/redesuas/lista-de-municipios-brasileiros/
#Codificação ISO-8859-1

from google.colab import files

print('Escolha o arquivo CSV')
ap_up = files.upload()
!mv *.csv arquivos

Agora é só unificar todos os arquivos CSV que você fez upload, os arquivos que eu estou utilizando nesse exemplo estão com uma codificação "ISO-8859-1", porém é possível subir arquivos com qualquer codificação, deixei comentado no código um outro padrão que é bem comum na internet "UTF-8":

In [ ]:
from base64 import encode

app = Core()
acervo = check_similar()

dir_list = app.dir_list(path=['arquivos'], extension='.csv$')

#Primeiro vamos unir todos os dicionários
if len(dir_list) > 0:
  for csvfile in dir_list:
    print(f'Trabalhando no arquivo {csvfile}')
    #app.read(input=csvfile, encode='UTF-8')
    app.read(input=csvfile, encode='ISO-8859-1')
  app.join()
  lista = app.export()

Informar na lista chamada "busca" as palavras que você deseja encontrar. No exemplo utilizado eu coloquei em uma frase completa com 2 municípios que estão na base CSV:

In [50]:
#Informar a coluna que está a informação que você deseja pesquisar
acervo.criar_lista(coluna='Município', df=lista)
#Na busca, você pode colocar em formato de lista todas buscas que deseja buscar
busca = ['Rio grande fica em qual estado?', 'Pontos turísticos de Boa Vista']

for nome in busca:
  #Retornar apenas valores com similaridade maior que 50%
  resultado = acervo.check_similar(busca=nome, porcentagem=50)
  if isinstance(resultado, dict):
    #Se retornar uma lista com mais de 1 resultado, mostrar valor em linhas
    for porcentagem in resultado:
       for valor in resultado[porcentagem]:
         print(f'{porcentagem}%: {valor}')
  elif resultado is None:
    #Se retornar "nada", informar que não encontrou nada
    print(f'0%: Não encontrado')
  else:
    #Se retornar um único valor, significar que foi um resultado 100%
    print(f'100%: {resultado}')

100%: RIO GRANDE
63.2%: BOA VISTA
51.6%: NOVA BOA VISTA
51.6%: ALTO BOA VISTA
